In [1]:
import os, sys
project_root = os.path.abspath(os.path.join(os.path.dirname('../')))
sys.path.append(project_root)

from deepface.basemodels import VGGFace
tf_face = VGGFace.loadModel()

from deepface.extendedmodels import Gender
tf_gender = Gender.loadModel()

from core.face.gender import Gender as PTGender
pt_gender = PTGender()

from core.face.vggface import VGGFace as PTVGGFace
torch_model = PTVGGFace()
# import torch
# from core.face.vggface import VGGFace as PTVGGFace
# torch_model = PTVGGFace()

# weights_file = os.path.join(project_root, "pretrained/vggface.pth")
# torch_model.load_state_dict(torch.load(weights_file))

2023-06-13 23:23:25.779097: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-13 23:23:25.800057: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 23:23:27.557327: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20548 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2023-06-13 23:23:27.557655: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device

In [2]:
import numpy as np
import torch
print(len(tf_face.layers), len(tf_gender.layers))

tf_gender_layer_id = 0
for i in range(len(tf_gender.layers)):  
  face_layer = tf_face.layers[i] if i < len(tf_face.layers) else None
  gender_layer =  tf_gender.layers[i]
  if face_layer: 
    pass
  #   print(face_layer.name, gender_layer.name)
  else:
  #   print(gender_layer.name)
    continue
  
  if not gender_layer.get_weights(): continue
  flatten_face_weights = face_layer.get_weights()[0].reshape(-1)
  flatten_gender_weights = gender_layer.get_weights()[0].reshape(-1)
  
  if not np.array_equal(flatten_face_weights, flatten_gender_weights):    
    print(face_layer.get_weights()[0].shape, gender_layer.get_weights()[0].shape)
    pt_gender_layer_id = 0
    for pt_gender_layer in pt_gender.modules():
      if isinstance(pt_gender_layer, torch.nn.Conv2d):
        if gender_layer.get_weights():
          if (pt_gender_layer_id == tf_gender_layer_id):
            weights = gender_layer.get_weights()
            pt_gender_layer.weight.data = torch.from_numpy(weights[0].transpose(3, 2, 0, 1))
            pt_gender_layer.bias.data = torch.from_numpy(weights[1])
            print(gender_layer)
            print(gender_layer.get_weights()[0].shape, pt_gender_layer.weight.data.shape)
            print(flatten_face_weights[:10])
            print(flatten_gender_weights[:10])
            print('------------------')
        pt_gender_layer_id += 1      
    tf_gender_layer_id += 1          
    
torch.save(pt_gender.state_dict(), os.path.join(project_root, "pretrained/gender.pth"))

38 39
(7, 7, 512, 4096) (7, 7, 512, 4096)
(7, 7, 512, 4096) torch.Size([4096, 512, 7, 7])
[-1.3083947e-04 -1.8606588e-05 -3.1403993e-05 -2.5138357e-05
  5.8494537e-05 -1.4254390e-03 -3.2078020e-05 -1.1948898e-03
 -1.5005474e-03 -1.1764151e-03]
[-4.2590671e-03 -1.8606588e-05 -1.1356508e-03 -2.5138357e-05
  5.8494537e-05 -1.4254390e-03 -3.2078020e-05 -1.9952398e-02
 -1.5005474e-03 -1.1306326e-02]
------------------
(1, 1, 4096, 4096) (1, 1, 4096, 4096)
(1, 1, 4096, 4096) torch.Size([4096, 4096, 1, 1])
[-7.9717307e-04  1.7204320e-04  3.0277763e-04 -1.1735294e-03
  1.1266565e-03  4.2761283e-04  5.6686375e-04 -1.0834980e-03
  8.9018205e-05 -2.5601126e-04]
[-0.00585147  0.00172603 -0.00489362 -0.00117353  0.00112666  0.00042761
  0.00056686  0.00153397 -0.00501941 -0.00025601]
------------------
(1, 1, 4096, 2622) (1, 1, 4096, 2)
(1, 1, 4096, 2) torch.Size([2, 4096, 1, 1])
[-0.01183689 -0.00843864 -0.00565795 -0.00274284 -0.01450934 -0.0028405
 -0.00491976 -0.00419261  0.00520571 -0.00645062

In [4]:
import torch
import cv2
import numpy as np

test_img = '/home/jixian/tmp/face_0_cropped.jpg'
test_img = '/home/jixian/tmp/3e6469e1b29ffa4e399cdd3b4e974a71914ca34c.jpg'
img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
tf_img = np.expand_dims(img, axis=0)
tf_img = tf_img.astype(np.float32)
tf_img /= 255.0
tf_res = tf_gender.predict(tf_img, verbose=0)[0].tolist()
print(tf_res)
tf_res = tf_gender.predict(tf_img, verbose=0)[0].tolist()
print(tf_res)
# convert img to torch tensor

img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
img = img.astype(np.float32)
img /= 255.0
img = img.transpose(2, 0, 1)
torch_img = torch.from_numpy(img)

pt_gender.eval()
with torch.no_grad():  
  features = torch_model.features(torch_img)
  # print(features.shape, features[:10])
  gender_probs = pt_gender(features)
  torch_res = gender_probs.detach().numpy().tolist()
  print('pt_gender:',torch_res)
    
  torch_res = pt_gender.predict(gender_probs)
  print(torch_res)

#   torch_res2 = torch_model(torch_img).reshape(-1).detach().numpy().tolist()
#   print(torch_res2[:10])

[0.9999594688415527, 4.055035242345184e-05]
[0.9999594688415527, 4.055035242345184e-05]
pt_gender: [0.9999594688415527, 4.0530670958105475e-05]
female


In [ ]:
total_params = sum(p.numel() for p in pt_gender.parameters())
print(f"Total number of parameters: {total_params}")
# for m in pt_gender.named_modules():
#   print(m[0])
  
for p in pt_gender.parameters():
  print(p.shape, p.numel())  

Total number of parameters: 119554050
torch.Size([4096, 512, 7, 7]) 102760448
torch.Size([4096]) 4096
torch.Size([4096, 4096, 1, 1]) 16777216
torch.Size([4096]) 4096
torch.Size([2, 4096, 1, 1]) 8192
torch.Size([2]) 2


In [ ]:
total_params = sum(p.numel() for p in torch_model.parameters())
print(f"Total number of parameters: {total_params}")
# for m in torch_model.named_modules():
#   print(m[0])
for p in torch_model.parameters():
  print(p.shape, p.numel())  

Total number of parameters: 145002878
torch.Size([64, 3, 3, 3]) 1728
torch.Size([64]) 64
torch.Size([64, 64, 3, 3]) 36864
torch.Size([64]) 64
torch.Size([128, 64, 3, 3]) 73728
torch.Size([128]) 128
torch.Size([128, 128, 3, 3]) 147456
torch.Size([128]) 128
torch.Size([256, 128, 3, 3]) 294912
torch.Size([256]) 256
torch.Size([256, 256, 3, 3]) 589824
torch.Size([256]) 256
torch.Size([256, 256, 3, 3]) 589824
torch.Size([256]) 256
torch.Size([512, 256, 3, 3]) 1179648
torch.Size([512]) 512
torch.Size([512, 512, 3, 3]) 2359296
torch.Size([512]) 512
torch.Size([512, 512, 3, 3]) 2359296
torch.Size([512]) 512
torch.Size([512, 512, 3, 3]) 2359296
torch.Size([512]) 512
torch.Size([512, 512, 3, 3]) 2359296
torch.Size([512]) 512
torch.Size([512, 512, 3, 3]) 2359296
torch.Size([512]) 512
torch.Size([4096, 512, 7, 7]) 102760448
torch.Size([4096]) 4096
torch.Size([4096, 4096, 1, 1]) 16777216
torch.Size([4096]) 4096
torch.Size([2622, 4096, 1, 1]) 10739712
torch.Size([2622]) 2622
